In [10]:
import pandas as pd
import os
import numpy as np

import warnings
warnings.filterwarnings('ignore')

In [12]:
# read extracted timeseries features
lvl2_train = pd.read_pickle("data/lvl2_imputer_train.pkl")
lvl2_dev =  pd.read_pickle("data/lvl2_imputer_dev.pkl")
lvl2_test =  pd.read_pickle("data/lvl2_imputer_test.pkl")

Ys =  pd.read_pickle("data/Ys.pkl")
Ys_train =  pd.read_pickle("data/Ys_train.pkl")
Ys_dev =  pd.read_pickle("data/Ys_dev.pkl")
Ys_test =  pd.read_pickle("data/Ys_test.pkl")

In [13]:
all_train_ids = set()
for i in Ys_train.itertuples():
    all_train_ids.add( i.Index[0] )

all_dev_ids = set()
for i in Ys_dev.itertuples():
    all_dev_ids.add( i.Index[0] )

all_test_ids = set()
for i in Ys_test.itertuples():
    all_test_ids.add( i.Index[0] )

In [14]:
print (sum(Ys_train.mort_icu.values)*1.0 / len(Ys_train.mort_icu.values))
print (sum(Ys_dev.mort_icu.values)*1.0 / len(Ys_dev.mort_icu.values))
print (sum(Ys_test.mort_icu.values)*1.0 / len(Ys_test.mort_icu.values))
print ("====")
print (sum(Ys_train.mort_hosp.values)*1.0 / len(Ys_train.mort_hosp.values))
print (sum(Ys_dev.mort_hosp.values)*1.0 / len(Ys_dev.mort_hosp.values))
print (sum(Ys_test.mort_hosp.values)*1.0 / len(Ys_test.mort_hosp.values))
print ("====")
print (sum(Ys_train.los_3.values)*1.0 / len(Ys_train.los_3.values))
print (sum(Ys_dev.los_3.values)*1.0 / len(Ys_dev.los_3.values))
print (sum(Ys_test.los_3.values)*1.0 / len(Ys_test.los_3.values))
print ("====")
print (sum(Ys_train.los_7.values)*1.0 / len(Ys_train.los_7.values))
print (sum(Ys_dev.los_7.values)*1.0 / len(Ys_dev.los_7.values))
print (sum(Ys_test.los_7.values)*1.0 / len(Ys_test.los_7.values))

0.07159904534606205
0.06725146198830409
0.07432150313152401
====
0.10680190930787589
0.09649122807017543
0.10855949895615867
====
0.43323389021479713
0.42105263157894735
0.4246346555323591
====
0.07696897374701671
0.07268170426065163
0.07954070981210856


### create time series data from word2vec embeddings

In [15]:
new_word2vec_dict = pd.read_pickle("data/new_ner_word2vec_dict.pkl")
new_keys = set(new_word2vec_dict.keys())
new_train_ids = sorted(all_train_ids.intersection(new_keys))
new_dev_ids = sorted(all_dev_ids.intersection(new_keys))
new_test_ids = sorted(all_test_ids.intersection(new_keys))

In [16]:
# save train, dev, test ids
type_of_ner = "new"
pd.to_pickle(new_train_ids, "data/"+type_of_ner+"_train_ids.pkl")
pd.to_pickle(new_dev_ids, "data/"+type_of_ner+"_dev_ids.pkl")
pd.to_pickle(new_test_ids, "data/"+type_of_ner+"_test_ids.pkl")


In [17]:
data_ids = [(new_train_ids, new_dev_ids, new_test_ids)]
data_names = ["new"]

for i, (tr, de, te) in zip(data_names, data_ids):

    y_train = Ys_train.loc[tr]
    y_dev = Ys_dev.loc[de]
    y_test = Ys_test.loc[te]

    sub_train = lvl2_train.loc[tr]
    sub_train = sub_train.loc[:, pd.IndexSlice[:, 'mean']]

    sub_dev = lvl2_dev.loc[de]
    sub_dev = sub_dev.loc[:, pd.IndexSlice[:, 'mean']]

    sub_test = lvl2_test.loc[te]
    sub_test = sub_test.loc[:, pd.IndexSlice[:, 'mean']]

    sub_train = sub_train.values
    sub_dev = sub_dev.values
    sub_test = sub_test.values

    # reshape the data for timeseries prediction
    x_train_lstm = sub_train.reshape(int(sub_train.shape[0] / 24), 24, 104)
    x_dev_lstm = sub_dev.reshape(int(sub_dev.shape[0] / 24), 24, 104)
    x_test_lstm = sub_test.reshape(int(sub_test.shape[0] / 24), 24, 104)


    pd.to_pickle(x_train_lstm, "data/"+i+"_x_train.pkl")
    pd.to_pickle(x_dev_lstm, "data/"+i+"_x_dev.pkl")
    pd.to_pickle(x_test_lstm, "data/"+i+"_x_test.pkl")

    pd.to_pickle(y_train, "data/"+i+"_y_train.pkl")
    pd.to_pickle(y_dev, "data/"+i+"_y_dev.pkl")
    pd.to_pickle(y_test, "data/"+i+"_y_test.pkl")

In [18]:
print(f"Final cohort (# of patients/labels) using Word2Vec: {len(y_train) + len(y_dev) + len(y_test)}")

Final cohort (# of patients/labels) using Word2Vec: 22203


### create time series data from fast text embeddings

In [5]:
new_fasttext_dict = pd.read_pickle("data/new_ner_fasttext_dict.pkl")
new_keys = set(new_fasttext_dict.keys())
new_train_ids_ft = sorted(all_train_ids.intersection(new_keys))
new_dev_ids_ft = sorted(all_dev_ids.intersection(new_keys))
new_test_ids_ft = sorted(all_test_ids.intersection(new_keys))

In [7]:
# save train, dev, test ids
type_of_ner = "new"
pd.to_pickle(new_train_ids_ft, "data/"+type_of_ner+"_train_ids_ft.pkl")
pd.to_pickle(new_dev_ids_ft, "data/"+type_of_ner+"_dev_ids_ft.pkl")
pd.to_pickle(new_test_ids_ft, "data/"+type_of_ner+"_test_ids_ft.pkl")

In [8]:
data_ids = [(new_train_ids_ft, new_dev_ids_ft, new_test_ids_ft)]
data_names = ["new"]

for i, (tr, de, te) in zip(data_names, data_ids):

    y_train = Ys_train.loc[tr]
    y_dev = Ys_dev.loc[de]
    y_test = Ys_test.loc[te]

    sub_train = lvl2_train.loc[tr]
    sub_train = sub_train.loc[:, pd.IndexSlice[:, 'mean']]

    sub_dev = lvl2_dev.loc[de]
    sub_dev = sub_dev.loc[:, pd.IndexSlice[:, 'mean']]

    sub_test = lvl2_test.loc[te]
    sub_test = sub_test.loc[:, pd.IndexSlice[:, 'mean']]

    sub_train = sub_train.values
    sub_dev = sub_dev.values
    sub_test = sub_test.values

    # reshape the data for timeseries prediction
    x_train_lstm = sub_train.reshape(int(sub_train.shape[0] / 24), 24, 104)
    x_dev_lstm = sub_dev.reshape(int(sub_dev.shape[0] / 24), 24, 104)
    x_test_lstm = sub_test.reshape(int(sub_test.shape[0] / 24), 24, 104)


    pd.to_pickle(x_train_lstm, "data/"+i+"_x_train_ft.pkl")
    pd.to_pickle(x_dev_lstm, "data/"+i+"_x_dev_ft.pkl")
    pd.to_pickle(x_test_lstm, "data/"+i+"_x_test_ft.pkl")

    pd.to_pickle(y_train, "data/"+i+"_y_train_ft.pkl")
    pd.to_pickle(y_dev, "data/"+i+"_y_dev_ft.pkl")
    pd.to_pickle(y_test, "data/"+i+"_y_test_ft.pkl")

In [9]:
print(f"Final cohort (# of patients/labels using Fast Text): {len(y_train) + len(y_dev) + len(y_test)}")

Final cohort (# of patients/labels using Fast Text): 22025


### create time series data from concatenated embeddings

In [20]:
new_concat_dict = pd.read_pickle("data/new_ner_combined_dict.pkl")
new_keys = set(new_concat_dict.keys())
new_train_ids_concat = sorted(all_train_ids.intersection(new_keys))
new_dev_ids_concat = sorted(all_dev_ids.intersection(new_keys))
new_test_ids_concat = sorted(all_test_ids.intersection(new_keys))

In [21]:
# save train, dev, test ids
type_of_ner = "new"
pd.to_pickle(new_train_ids_concat, "data/"+type_of_ner+"_train_ids_concat.pkl")
pd.to_pickle(new_dev_ids_concat, "data/"+type_of_ner+"_dev_ids_concat.pkl")
pd.to_pickle(new_test_ids_concat, "data/"+type_of_ner+"_test_ids_concat.pkl")

In [22]:
data_ids = [(new_train_ids_concat, new_dev_ids_concat, new_test_ids_concat)]
data_names = ["new"]

for i, (tr, de, te) in zip(data_names, data_ids):

    y_train = Ys_train.loc[tr]
    y_dev = Ys_dev.loc[de]
    y_test = Ys_test.loc[te]

    sub_train = lvl2_train.loc[tr]
    sub_train = sub_train.loc[:, pd.IndexSlice[:, 'mean']]

    sub_dev = lvl2_dev.loc[de]
    sub_dev = sub_dev.loc[:, pd.IndexSlice[:, 'mean']]

    sub_test = lvl2_test.loc[te]
    sub_test = sub_test.loc[:, pd.IndexSlice[:, 'mean']]

    sub_train = sub_train.values
    sub_dev = sub_dev.values
    sub_test = sub_test.values

    # reshape the data for timeseries prediction
    x_train_lstm = sub_train.reshape(int(sub_train.shape[0] / 24), 24, 104)
    x_dev_lstm = sub_dev.reshape(int(sub_dev.shape[0] / 24), 24, 104)
    x_test_lstm = sub_test.reshape(int(sub_test.shape[0] / 24), 24, 104)


    pd.to_pickle(x_train_lstm, "data/"+i+"_x_train_concat.pkl")
    pd.to_pickle(x_dev_lstm, "data/"+i+"_x_dev_concat.pkl")
    pd.to_pickle(x_test_lstm, "data/"+i+"_x_test_concat.pkl")

    pd.to_pickle(y_train, "data/"+i+"_y_train_concat.pkl")
    pd.to_pickle(y_dev, "data/"+i+"_y_dev_concat.pkl")
    pd.to_pickle(y_test, "data/"+i+"_y_test_concat.pkl")

In [23]:
print(f"Final cohort (# of patients/labels using concatenated embeddings): {len(y_train) + len(y_dev) + len(y_test)}")

Final cohort (# of patients/labels using concatenated embeddings): 22446
